* movie reviews NLTK movie_reviews
* subjectivitiydataset: nltk subjectivity

In [1]:
import torch
from torchtext.datasets import IMDB
from torchtext import data, datasets
import os
import pandas as pd
import spacy
# Let's use simple Naive Bayes Classification
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.svm import SVC

vectorizer = CountVectorizer()
classifier = MultinomialNB()


In [2]:
nlp = spacy.load("en_core_web_lg")
# nlp.add_pipe("sentencizer")
# print(nlp.pipeline)


In [3]:

SEED = 5
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
training_set=IMDB(split="train")
testing_set=IMDB(split="test")

In [5]:
type(training_set)

torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe

In [6]:
train_labels = []
train_texts  = []
valid_texts  = []
valid_labels = []
####################
train_text_pos = []
train_label_pos = []
train_text_neg = []
train_label_neg = []
for label, text in training_set:
  if label == 'pos':
    train_label_pos.append(1)
    train_text_pos.append(text)
  else:
    train_label_neg.append(0)
    train_text_neg.append(text)


In [7]:
valid_texts=train_text_pos[0:2500]+train_text_neg[0:2500]

In [8]:
valid_labels=train_label_pos[0:2500]+train_label_neg[0:2500]

In [9]:
train_texts=train_text_pos[2500:]+train_text_neg[2500:]

In [10]:
train_labels=train_label_pos[2500:]+train_label_neg[2500:]

In [11]:
len(train_texts)

20000

In [12]:
len(train_texts)

20000

In [13]:
test_labels = []
test_texts = []
for label, text in testing_set:
  if label == 'pos':
    test_labels.append(1)
  else:
    test_labels.append(0)
  test_texts.append(text)

In [14]:
len(test_texts)

25000

In [15]:
df_train=pd.DataFrame({'review': train_texts, 'label':train_labels})
df_valid=pd.DataFrame({'review': valid_texts, 'label':valid_labels})
df_test=pd.DataFrame({'review': test_texts, 'label':test_labels})

In [16]:
df_train[df_train.duplicated()] # we have 77 item duplicated out of 10k

,review,label
644,I loved this movie. I knew it would be chocked...,1
1812,"During a Kurt Weill celebration in Brooklyn, W...",1
1821,I really enjoyed this movie. The script is fre...,1
3158,I'm gonna tip the scales here a bit and say I ...,1
3624,I am decidedly not in the target audience for ...,1
...,...,...
19810,"When i got this movie free from my job, along ...",0
19883,No spoilers here but I have been a fan since W...,0
19884,I feel like I've just watched a snuff film.......,0
19885,"I absolutely hate this programme, what kind of...",0


In [17]:
df_test[df_test.duplicated()] # we 199 itmes duplicates out of 25k

,review,label
861,The movie was very good. I'm an avid mystery f...,1
908,I've been strangely attracted to this film sin...,1
919,I was fortunate to attend the London premier o...,1
1510,A truly excellent look at the world and the re...,1
1520,Robert Jordan is a television star. Robert Jor...,1
...,...,...
24146,"Poorly acted, poorly written and poorly direct...",0
24147,Ed Wood rides again. The fact that this movie ...,0
24266,This movie looked like it was rushed to releas...,0
24401,I kind of like JAG. It do have it´s charm but ...,0


In [18]:
df_valid[df_valid.duplicated()] # we have 8 item duplicated out of 2.5k

,review,label
763,A have a female friend who is currently being ...,1
924,"This movie surprised me. Some things were ""cli...",1
947,Wow! So much fun! Probably a bit much for norm...,1
1380,I Enjoyed Watching This Well Acted Movie Very ...,1
1642,Smallville episode Justice is the best episode...,1
2668,I am not so much like Love Sick as I image. Fi...,0
3164,Holy freaking God all-freaking-mighty. This mo...,0
3201,"The story and the show were good, but it was r...",0


In [19]:
vectorizer = CountVectorizer()
classifier = MultinomialNB()
classifier_SVM = SVC()


In [20]:
corpus =train_texts+valid_texts+test_texts
label =train_labels+valid_labels+test_labels
vectors = vectorizer.fit_transform(corpus)
train_vectors = vectors[:len(train_texts)]
valid_vectors = vectors[len(train_texts):len(train_texts)+len(valid_texts)]
test_vectors = vectors[len(train_texts)+len(valid_texts):]

In [21]:
print(train_vectors.shape)
print(valid_vectors.shape)
print(test_vectors.shape)

(20000, 101895)
(5000, 101895)
(25000, 101895)


In [22]:
scores = cross_validate(classifier, vectors, label, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print(round(average, 3))


0.807


In [ ]:
# scores = cross_validate(classifier_SVM, vectors, label, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
# average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
# print(round(average, 3))


## Baseline Vader for document level sentiment polarity classifcation


In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer, VaderConstants
analyzer = SentimentIntensityAnalyzer()

In [32]:
scores = [analyzer.polarity_scores(docs) for docs in train_texts]
labels = [(0 if s.get("compound")<0 else 1)for s in scores]
print(classification_report(train_labels, labels, digits=3))

              precision    recall  f1-score   support

           0      0.779     0.537     0.636     10000
           1      0.647     0.848     0.734     10000

    accuracy                          0.692     20000
   macro avg      0.713     0.692     0.685     20000
weighted avg      0.713     0.692     0.685     20000



In [33]:
scores = [analyzer.polarity_scores(docs) for docs in test_texts]
labels = [(0 if s.get("compound")<0 else 1)for s in scores]
print(classification_report(test_labels, labels, digits=3))

              precision    recall  f1-score   support

           0      0.793     0.536     0.640     12500
           1      0.650     0.860     0.740     12500

    accuracy                          0.698     25000
   macro avg      0.722     0.698     0.690     25000
weighted avg      0.722     0.698     0.690     25000

